# Analisador Sintático Baseado em Autômatos Finitos - Parte 2: Implementação do Autômato

Nesta parte, vamos explorar a implementação do autômato finito usado para a análise sintática.

In [2]:
# Importar bibliotecas necessárias
import sys
sys.path.append('.')

from automaton import FiniteAutomaton, State
from syntactic_analyzer import SyntacticAnalyzer

## 1. Estrutura do Autômato

O autômato finito é implementado na classe `FiniteAutomaton` no arquivo `automaton.py`. Esta classe possui:

- Uma coleção de estados (`states`)
- Um estado inicial (`initial_state`)
- Um estado atual (`current_state`)
- Métodos para adicionar estados e transições
- Métodos para processar símbolos de entrada

Cada estado é representado pela classe `State`, que possui:

- Um nome (`name`)
- Um indicador se é um estado final (`is_final`)
- Um dicionário de transições (`transitions`)

In [3]:
# Criar um autômato simples para demonstração
automaton = FiniteAutomaton("DemoAutomaton")

# Adicionar estados
automaton.add_state("q0")
automaton.add_state("q1")
automaton.add_state("q2", is_final=True)

# Definir estado inicial
automaton.set_initial_state("q0")

# Adicionar transições
automaton.add_transition("q0", "a", "q1")
automaton.add_transition("q1", "b", "q2")

# Testar o autômato
print(f"Autômato: {automaton}")
print(f"Processando 'ab': {automaton.process_string('ab')}")
print(f"Processando 'abc': {automaton.process_string('abc')}")
print(f"Processando 'ba': {automaton.process_string('ba')}")

Autômato: Automaton(DemoAutomaton, states=[q0, q1, q2*], initial=q0)
Processando 'ab': True
Processando 'abc': False
Processando 'ba': False


## 2. Autômato para Análise Sintática

O autômato específico para análise sintática é construído no método `_build_automaton` da classe `SyntacticAnalyzer`. Este autômato possui os seguintes estados:

- **q0**: Estado inicial
- **q1**: Após reconhecer determinante do sujeito
- **q2**: Após reconhecer sujeito
- **q3**: Após reconhecer verbo
- **q4**: Após reconhecer determinante do objeto
- **q5**: Após reconhecer objeto ou advérbio
- **q6**: Após reconhecer preposição
- **q7**: Após reconhecer determinante do objeto indireto
- **q8**: Após reconhecer objeto indireto
- **q9**: Estado final (após pontuação)

In [4]:
# Criar um analisador sintático
analyzer = SyntacticAnalyzer()

# Obter a descrição do autômato
automaton_description = analyzer.get_automaton_description()
print(automaton_description)

Autômato para Análise Sintática:

Estados:
  - q0: Estado inicial (início da frase)
  - q1: Após reconhecer determinante do sujeito
  - q2: Após reconhecer sujeito
  - q3: Após reconhecer verbo
  - q4: Após reconhecer determinante do objeto
  - q5: Após reconhecer objeto
  - q6: Após reconhecer preposição
  - q7: Após reconhecer determinante do objeto indireto
  - q8: Após reconhecer objeto indireto
  - q9: Estado final (após pontuação)

Transições:
  - q0 --DET_SUJEITO--> q1
  - q0 --SUJEITO--> q2
  - q1 --SUJEITO--> q2
  - q2 --VERBO--> q3
  - q3 --DET_OBJETO--> q4
  - q3 --OBJETO--> q5
  - q3 --PONTUACAO--> q9
  - q4 --OBJETO--> q5
  - q5 --PREPOSICAO--> q6
  - q5 --PONTUACAO--> q9
  - q6 --DET_OBJ_IND--> q7
  - q6 --OBJ_INDIRETO--> q8
  - q7 --OBJ_INDIRETO--> q8
  - q8 --PONTUACAO--> q9

Representação Visual:

        (q0) --DET_SUJEITO--> (q1) --SUJEITO--> (q2) --VERBO--> (q3) --DET_OBJETO--> (q4) --OBJETO--> (q5) --PREPOSICAO--> (q6) --DET_OBJ_IND--> (q7) --OBJ_INDIRETO--> (q8) -

## 3. Transições do Autômato

As transições entre estados são definidas com base nas funções sintáticas:

- q0 → q1: DET_SUJEITO (determinante do sujeito)
- q0 → q2: SUJEITO (sujeito sem determinante)
- q1 → q2: SUJEITO (sujeito após determinante)
- q2 → q3: VERBO
- q3 → q4: DET_OBJETO (determinante do objeto)
- q3 → q5: OBJETO/PREDICATIVO/VERBO_INFINITIVO/ADVERBIO
- q3 → q6: PREPOSICAO (preposição após verbo)
- q3 → q9: PONTUACAO (frase termina após verbo)
- q4 → q5: OBJETO (objeto após determinante)
- q5 → q5: ADVERBIO (advérbio após objeto ou outro advérbio)
- q5 → q6: PREPOSICAO (preposição após objeto)
- q5 → q9: PONTUACAO (frase termina após objeto)
- q6 → q7: DET_OBJ_IND (determinante do objeto indireto)
- q6 → q8: OBJ_INDIRETO (objeto indireto sem determinante)
- q7 → q8: OBJ_INDIRETO (objeto indireto após determinante)
- q8 → q8: ADVERBIO (advérbio após objeto indireto)
- q8 → q9: PONTUACAO (frase termina após objeto indireto)

In [5]:
# Visualizar o código que constrói o autômato
import inspect

build_automaton_code = inspect.getsource(SyntacticAnalyzer._build_automaton)
print(build_automaton_code)

    def _build_automaton(self):
        """
        Constrói um autômato finito para análise sintática.
        Este autômato identifica funções sintáticas básicas como sujeito, verbo e objeto.
        """
        # Criar autômato para análise sintática
        automaton = FiniteAutomaton("SyntacticAnalyzer")
        
        # Estados do autômato
        # q0: Estado inicial
        # q1: Após reconhecer determinante do sujeito
        # q2: Após reconhecer sujeito
        # q3: Após reconhecer verbo
        # q4: Após reconhecer determinante do objeto
        # q5: Após reconhecer objeto ou advérbio
        # q6: Após reconhecer preposição
        # q7: Após reconhecer determinante do objeto indireto
        # q8: Após reconhecer objeto indireto
        # q9: Estado final (após pontuação)
        
        # Adicionar estados
        for i in range(10):
            is_final = (i == 9)  # Apenas q9 é estado final
            automaton.add_state(f"q{i}", is_final=is_final)
        
    

## 4. Processamento de Entrada pelo Autômato

Durante a análise sintática, o autômato processa uma sequência de funções sintáticas (não as palavras em si). O método `analyze_sentence` da classe `SyntacticAnalyzer` coordena este processo:

1. Tokeniza a frase
2. Classifica cada token em uma categoria gramatical
3. Mapeia cada token para uma função sintática
4. Processa a sequência de funções sintáticas pelo autômato
5. Verifica se o autômato termina em um estado final

Se o autômato terminar em um estado final, a frase é considerada sintaticamente válida.

In [6]:
# Analisar uma frase e mostrar o caminho percorrido no autômato
frase = "Eu liguei ao diretor ontem."
is_valid, syntactic_tokens, path = analyzer.analyze_sentence(frase)

print(f"Frase: '{frase}'")
print(f"Estrutura sintática válida: {is_valid}\n")

print("Caminho percorrido no autômato:")
for i, (state, symbol) in enumerate(path):
    if i == 0:
        print(f"  {state} (estado inicial)")
    else:
        print(f"  {state} <-- {symbol} --")

Frase: 'Eu liguei ao diretor ontem.'
Estrutura sintática válida: True

Caminho percorrido no autômato:
  q0 (estado inicial)
  q2 <-- SUJEITO --
  q3 <-- VERBO --
  q6 <-- PREPOSICAO --
  q8 <-- OBJ_INDIRETO --
  q8 <-- ADVERBIO --
  q9 <-- PONTUACAO --
